In [72]:
# filelist
import cv2
import numpy as np
import os
from pathlib import Path
# root = os.getcwd() # Don't change this
# data_dirname = '/data_test/plantvillage/' # Change as you like 
# p = Path(root + data_dirname)
# p.mkdir(exist_ok=True) 

p = Path("C:/Users/mwels/Documents/Uni/11. Semester/Deep learning in visual recognition/Plant_leave_diseases_dataset_without_augmentation")
p.mkdir(exist_ok=True)


classes = [
  'Apple___Apple_scab',
  'Apple___healthy',
  'Apple___Black_rot',
  'Apple___Cedar_apple_rust',
  "all"]

if "all" in classes:
  classes = os.listdir(p)

for c in classes:
  print(c,end=" ")
  filelist = [x for x in (p/c).iterdir() if x.is_file()]
  for f in filelist:
    img = cv2.imread(str(f))
    if img is None:
      print(f'Failed to open {f}. Deleting file')
      os.remove(str(f))
# Pick first image of first class (i.e., cat)
# filelist = [x for x in (p/classes[0]).iterdir() if x.is_file()]
print(len(filelist))

Apple___Apple_scab Apple___Black_rot Apple___Cedar_apple_rust Apple___healthy Background_without_leaves Blueberry___healthy Cherry___healthy Cherry___Powdery_mildew Corn___Cercospora_leaf_spot Gray_leaf_spot Corn___Common_rust Corn___healthy Corn___Northern_Leaf_Blight Grape___Black_rot Grape___Esca_(Black_Measles) Grape___healthy Grape___Leaf_blight_(Isariopsis_Leaf_Spot) Orange___Haunglongbing_(Citrus_greening) Peach___Bacterial_spot Peach___healthy Pepper,_bell___Bacterial_spot Pepper,_bell___healthy Potato___Early_blight Potato___healthy Potato___Late_blight Raspberry___healthy Soybean___healthy Squash___Powdery_mildew Strawberry___healthy Strawberry___Leaf_scorch Tomato___Bacterial_spot Tomato___Early_blight Tomato___healthy Tomato___Late_blight Tomato___Leaf_Mold Tomato___Septoria_leaf_spot Tomato___Spider_mites Two-spotted_spider_mite Tomato___Target_Spot Tomato___Tomato_mosaic_virus Tomato___Tomato_Yellow_Leaf_Curl_Virus 5357


In [41]:
from keras.applications.mobilenet import MobileNet
conv_base = MobileNet(weights='imagenet',
                      include_top=False,
                      input_shape=(40, 40, 3))
type(conv_base)

keras.engine.functional.Functional

In [74]:
from keras.layers import Dense,GlobalAveragePooling2D, Concatenate, Input, Flatten, Lambda
from keras.models import Model
# from src.fast_soft_sort.tf_ops import soft_rank


number_of_tiles = 9
image_net = conv_base
tiles = Input((9,40,40,3))
tile1 = Lambda(lambda x: x[:,0,:,:,:])(tiles) 
tile2 = Lambda(lambda x: x[:,1,:,:,:])(tiles) 
tile3 = Lambda(lambda x: x[:,2,:,:,:])(tiles) 
tile4 = Lambda(lambda x: x[:,3,:,:,:])(tiles) 
tile5 = Lambda(lambda x: x[:,4,:,:,:])(tiles) 
tile6 = Lambda(lambda x: x[:,5,:,:,:])(tiles) 
tile7 = Lambda(lambda x: x[:,6,:,:,:])(tiles) 
tile8 = Lambda(lambda x: x[:,7,:,:,:])(tiles) 
tile9 = Lambda(lambda x: x[:,8,:,:,:])(tiles) 

embedding_of_tile1 = image_net(tile1) 
embedding_of_tile2 = image_net(tile2)
embedding_of_tile3 = image_net(tile3) 
embedding_of_tile4 = image_net(tile4) 
embedding_of_tile5 = image_net(tile5) 
embedding_of_tile6 = image_net(tile6)
embedding_of_tile7 = image_net(tile7)
embedding_of_tile8 = image_net(tile8)
embedding_of_tile9 = image_net(tile9)

concatonation = Concatenate(axis=3)([
    embedding_of_tile1,
    embedding_of_tile2,
    embedding_of_tile3,
    embedding_of_tile4,
    embedding_of_tile5,
    embedding_of_tile6,
    embedding_of_tile7,
    embedding_of_tile8,
    embedding_of_tile9
    ])

# out = Dense(500, activation="relu")(concatonation)
out = Dense(500, activation="relu")(out)
out = Dense(9)(out)
out = Flatten()(out)
# out = Lambda(lambda x: soft_rank(x, regularization_strength=0.5))(out)

model = Model(inputs=tiles, outputs=out)

In [75]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 9, 40, 40,   0           []                               
                                3)]                                                               
                                                                                                  
 lambda_27 (Lambda)             (None, 40, 40, 3)    0           ['input_6[0][0]']                
                                                                                                  
 lambda_28 (Lambda)             (None, 40, 40, 3)    0           ['input_6[0][0]']                
                                                                                                  
 lambda_29 (Lambda)             (None, 40, 40, 3)    0           ['input_6[0][0]']          

In [44]:
total_num_layers = len(model.layers)
num_base_layers = len(conv_base.layers)
print(f"Total number of layers is {total_num_layers}")
print(f"Number of pretrained base layers is {num_base_layers}")

for layer in model.layers[:num_base_layers]:
    layer.trainable=False
for layer in model.layers[num_base_layers:]:
    layer.trainable=True

Total number of layers is 15
Number of pretrained base layers is 86


In [76]:
from keras import optimizers
from sklearn.model_selection import train_test_split
from src.permutation_tools import *
import tensorflow_ranking as tfr 

x_train, x_test = train_test_split(filelist)

train_generator = PermNetDataGenerator(x_train)
validation_generator = PermNetDataGenerator(x_test)


# Set up optimizer
sgd_optimizer = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)

# Compile model 
# softmax_loss = tfr.keras.losses.get(tfr.keras.losses.RankingLossKey.MEAN_SQUARED_LOSS)
model.compile(optimizer=sgd_optimizer,loss=tfr.keras.losses.MeanSquaredLoss(), metrics=['accuracy'])

step_size_train = train_generator.n//train_generator.batch_size # Number of mini-batches per epoch (training)
step_size_val = validation_generator.n//validation_generator.batch_size # Number of mini-batches per epoch (validation)

# Train model for 15 epochs
history = model.fit_generator(generator=train_generator,
                   validation_data=validation_generator,
                   validation_steps=step_size_val,
                   steps_per_epoch=step_size_train,
                   epochs=15)

C:\Users\mwels\AppData\Local\Temp\ipykernel_6060\2570840799.py:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_generator,


Epoch 1/15
62/62 [==============================] - 77s 1s/step - loss: 15.7536 - accuracy: 0.1085 - val_loss: 7.5214 - val_accuracy: 0.1219
Epoch 2/15
62/62 [==============================] - 52s 836ms/step - loss: 7.0949 - accuracy: 0.1133 - val_loss: 7.0642 - val_accuracy: 0.1000
Epoch 3/15
62/62 [==============================] - 57s 924ms/step - loss: 7.0360 - accuracy: 0.1305 - val_loss: 7.0460 - val_accuracy: 0.1312
Epoch 4/15
62/62 [==============================] - 56s 906ms/step - loss: 7.0232 - accuracy: 0.1272 - val_loss: 7.0319 - val_accuracy: 0.1234
Epoch 5/15
62/62 [==============================] - 67s 1s/step - loss: 7.0164 - accuracy: 0.1361 - val_loss: 7.0270 - val_accuracy: 0.1281
Epoch 6/15
62/62 [==============================] - 58s 940ms/step - loss: 7.0019 - accuracy: 0.1235 - val_loss: 7.0151 - val_accuracy: 0.1477
Epoch 7/15
62/62 [==============================] - 61s 974ms/step - loss: 6.9893 - accuracy: 0.1374 - val_loss: 7.0070 - val_accuracy: 0.1383
Epoc

4.6666665

In [49]:
len(filelist)

630